## Optimização Heurística

### **Licenciatura em Ciência de Dados**

### **Projeto Grupo**

CDB1


* João Batista, Nº 111611
* João Dias, Nº110305
* António Teotónio, Nº 111283
* Diogo Aqueu, Nº 110705

**Docentes:** Anabela Costa / Mafalda Pontes

**Data:**  3 de junho de 2024

### Setup inicial e load de bibliotecas

In [1]:
import random
import numpy as np
import pandas as pd

### Implementação Algoritmo Genético

In [2]:
# Parâmetros do algoritmo
POP_SIZE = 200
PC = 0.8  # Probabilidade de crossover
PM = 0.1  # Probabilidade de mutação
MAX_GEN = 1000
MAX_TIME = 480  # 8 horas = 480 minutos
MAX_ITER = 100  # Número máximo de iterações
MAX_ITER_NO_IMPROVEMENT = 15  # Limite de iterações sem melhoria

# Categorias dos enfermeiros (1: categoria 1, 2: categoria 2, 3: categoria 3)
CATEGORIES = {
    "E1": 1, "E2": 1, "E3": 1, "E4": 1, "E5": 2, "E6": 2, "E7": 2, "E8": 2, "E9": 3, "E10": 3
}

# Procedimentos complexos (índices dos procedimentos complexos no cromossoma)
COMPLEX_PROCEDURES = [6,7,8,18,19,20,21,22,23,27,28,29,33,34,35]

# Duração dos procedimentos 
durations = pd.read_excel('Trab_Grupo.xlsx').to_numpy()

# Converte enfermeiros para índices para fácil manipulação
nurses = list(CATEGORIES.keys())
nurse_to_idx = {nurse: idx for idx, nurse in enumerate(nurses)}


def generate_initial_population():
    population = []
    for _ in range(POP_SIZE):
        chromosome = []
        for _ in range(7):  # 7 períodos
            period = random.sample(nurses, 6)  # 6 enfermeiros por período, sem repetição
            chromosome.extend(period)
        if not is_admissible(chromosome):
            chromosome = repair(chromosome)
        population.append(chromosome)
    return population

def evaluate(chromosome):
    total_duration = 0
    for period in range(7):
        period_procedures = chromosome[period * 6:(period + 1) * 6]
        if period >= 1:
            period = period + period
        proc1_durations = [durations[period][int(s[1:])-1] for s in period_procedures[:3]]
        proc2_durations = [durations[period+1][int(s[1:])-1] for s in period_procedures[3:]]
        if period > 1:
            period = period - 1
        max_duration = max(max(proc1_durations), max(proc2_durations))
        total_duration += max_duration
    return total_duration

def selection(population, fitnesses):
    selected = []
    for _ in range(POP_SIZE // 2):
        tournament = random.sample(range(POP_SIZE), 2)
        winner = tournament[0] if fitnesses[tournament[0]] < fitnesses[tournament[1]] else tournament[1]
        selected.append(population[winner])
    return selected

def crossover(parent1, parent2):
    if random.random() < PC:
        m = random.randint(1, len(parent1) - 1)
        child1 = parent1[:m] + parent2[m:]
        child2 = parent2[:m] + parent1[m:]
        if not is_admissible(child1):
            child1 = repair(child1)  # Corrigir possíveis duplicados
        if not is_admissible(child2):
            child2 = repair(child2)  # Corrigir possíveis duplicados
        return [child1, child2]
    else:
        if not is_admissible(parent1):
            parent1 = repair(parent1)  
        if not is_admissible(parent2):
            parent2 = repair(parent2) 
        return [parent1, parent2]

def mutate(chromosome):
    mutated_chromosome = chromosome[:]  # Cria uma cópia do cromossoma original
    if random.random() < PM:
        i, j = random.sample(range(len(mutated_chromosome)), 2)
        mutated_chromosome[i], mutated_chromosome[j] = mutated_chromosome[j], mutated_chromosome[i]
    if not is_admissible(mutated_chromosome):
        mutated_chromosome = repair(mutated_chromosome)  # Corrige o cromossoma mutado
    return mutated_chromosome

def is_admissible(chromosome):
    nurse_counts = {nurse: 0 for nurse in nurses}
    for period in range(7):
        period_procedures = chromosome[period * 6:(period + 1) * 6]
        for nurse in period_procedures:
            nurse_counts[nurse] += 1
            if nurse_counts[nurse] > 5:
                return False  # Enfermeiro aparece em mais de 5 períodos
        if len(period_procedures) != len(set(period_procedures)):
            return False  # Enfermeiro duplicado no mesmo período
        for procedure_idx in COMPLEX_PROCEDURES:
            nurse = chromosome[procedure_idx]
            if CATEGORIES[nurse] == 1:
                return False  # Enfermeiro de categoria 1 designado para procedimento complexo
    return True

def repair(chromosome):
    max_iterations = 100  # Define um limite máximo de iterações sem alterações
    iteration_count = 0
    
    while not is_admissible(chromosome):
        for period in range(7):
            period_procedures = chromosome[period * 6:(period + 1) * 6]
            unique_nurses = []
            for nurse in period_procedures:
                if nurse not in unique_nurses:
                    unique_nurses.append(nurse)
            # Preencher os períodos com enfermeiros adicionais 
            while len(unique_nurses) < 6:
                available_nurses = [n for n in nurses if n not in unique_nurses]
                if not available_nurses:
                    break
                nurse_to_add = random.choice(available_nurses)
                unique_nurses.append(nurse_to_add)
            chromosome[period * 6:(period + 1) * 6] = unique_nurses

        nurse_counts = {nurse: 0 for nurse in nurses}
        for period in range(7):
            period_procedures = chromosome[period * 6:(period + 1) * 6]
            for nurse in period_procedures:
                nurse_counts[nurse] += 1

        for nurse, count in nurse_counts.items():
            if count > 5:
                for _ in range(count - 5):
                    for period in range(7):
                        if chromosome[period * 6] == nurse:
                            chromosome[period * 6] = random.choice([n for n in nurses if n != nurse])

        for procedure_idx in COMPLEX_PROCEDURES:
            nurse = chromosome[procedure_idx]
            if CATEGORIES[nurse] == 1:
                available_nurses = [n for n in nurses if CATEGORIES[n] != 1]
                chromosome[procedure_idx] = random.choice(available_nurses)
        
        iteration_count += 1
        if iteration_count > max_iterations:
            # Se atingir o limite de iterações sem alterações, reinicia o cromossoma 
            def initialize_chromosome():
                chromosome = []
                for _ in range(7):  # 7 períodos
                    for _ in range(6):  # 6 enfermeiros por período
                        nurse = random.choice(nurses)  # Seleciona um enfermeiro aleatório
                        chromosome.append(nurse)
                return chromosome
            chromosome = initialize_chromosome()  # Reinicializar o cromossoma
            iteration_count = 0  # Reiniciar o contador de iterações
    
    while not is_admissible(chromosome):
        repair(chromosome)
    
    return chromosome

def genetic_algorithm():
    population = generate_initial_population()
    best_solution = None
    best_fitness = float('inf')
    iter_without_improvement = 0

    for generation in range(MAX_GEN):
        fitnesses = [evaluate(chromosome) for chromosome in population]
        min_fitness = min(fitnesses)
        min_index = fitnesses.index(min_fitness)

        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_solution = population[min_index]
            iter_without_improvement = 0
        else:
            iter_without_improvement += 1

        # Se não houver melhoria por muitas iterações, interromper o algoritmo
        if iter_without_improvement >= MAX_ITER_NO_IMPROVEMENT:
            print('Algoritmo parou ao fim de ', iter_without_improvement, ' iterações sem melhorias significativas')
            break

        # Imprimir os cromossomas testados e os seus valores de duração
        print("Geração:", generation)
        for i, chromosome in enumerate(population):
            print("Cromossoma:", chromosome, "Duração:", fitnesses[i])

        if min_fitness <= MAX_TIME:
            print('Algoritmo parou por se obter ', min_fitness)
            break

        if generation >= MAX_ITER:
            print('Algoritmo parou ao fim de ', generation , ' iterações')
            break

        selected = selection(population, fitnesses)
        next_population = []

        while len(next_population) < POP_SIZE:
            parents = random.sample(selected, 2)
            offspring = crossover(parents[0], parents[1])
            for child in offspring:
                mutated_child = mutate(child)
                next_population.append(mutated_child)

        population = next_population

    return best_solution, best_fitness

# Executar o Algoritmo Genético
random.seed(123)
best_solution, best_fitness = genetic_algorithm()
print("Melhor solução encontrada:", best_solution)
print("Duração total:", best_fitness)

Geração: 0
Cromossoma: ['E6', 'E2', 'E4', 'E5', 'E3', 'E7', 'E5', 'E10', 'E9', 'E2', 'E1', 'E8', 'E6', 'E8', 'E7', 'E3', 'E4', 'E9', 'E10', 'E8', 'E6', 'E9', 'E5', 'E7', 'E3', 'E9', 'E4', 'E7', 'E6', 'E5', 'E3', 'E5', 'E10', 'E6', 'E7', 'E9', 'E3', 'E1', 'E4', 'E8', 'E10', 'E2'] Duração: 571
Cromossoma: ['E8', 'E6', 'E9', 'E3', 'E10', 'E7', 'E6', 'E10', 'E7', 'E2', 'E5', 'E1', 'E7', 'E2', 'E8', 'E6', 'E1', 'E5', 'E8', 'E10', 'E9', 'E6', 'E5', 'E7', 'E3', 'E4', 'E2', 'E10', 'E9', 'E8', 'E1', 'E10', 'E2', 'E9', 'E8', 'E5', 'E1', 'E5', 'E6', 'E2', 'E4', 'E9'] Duração: 580
Cromossoma: ['E9', 'E1', 'E6', 'E5', 'E3', 'E10', 'E7', 'E10', 'E6', 'E4', 'E8', 'E3', 'E1', 'E9', 'E3', 'E2', 'E8', 'E4', 'E9', 'E7', 'E8', 'E5', 'E10', 'E6', 'E2', 'E4', 'E8', 'E9', 'E7', 'E5', 'E6', 'E3', 'E7', 'E8', 'E10', 'E5', 'E10', 'E5', 'E4', 'E7', 'E6', 'E1'] Duração: 612
Cromossoma: ['E2', 'E6', 'E3', 'E8', 'E9', 'E7', 'E8', 'E9', 'E10', 'E3', 'E1', 'E6', 'E1', 'E7', 'E4', 'E9', 'E2', 'E3', 'E5', 'E10', 'E7', 